<a href="https://colab.research.google.com/github/Dunya18/Operational_Research/blob/main/TP2_RO_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 Operational research

## If you have Cplex-Python installation problem 

Step 1: Check if your python version is **3.7**

In [ ]:
import sys
print(sys.version)
#print(sys.executable)

3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]


If not,\
$~~$    install Python 3.7 directly and run jupyter notebook with python 3.7 \
or\
$~~$    create a virtual python 3.7 enrivoments by anaconda (**recommended**) :  see https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html or other instructional sources by Google

Step 2: run the following command 

In [ ]:
!pip install cplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 15.9 MB/s eta 0:00:00


In [ ]:
!pip install docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671364 sha256=12e3d9375d6dc3492681b2f4758a7b306c27c89714993d2dde0829f243ef2b02
  Stored in directory: /root/.cache/pip/wheels/b8/98/f8/22c3fe8d29be988cc4584363f494a459fb8f09c16d8e438ac7
Successfully built docplex


## Integer programming 

In [ ]:
from docplex.mp.model import Model

Another example : model of the telephone production problem
    
* Decision variables:
    * Number of desk phones produced (DeskProduction)
    * Number of cellular phones produced (CellProduction)
* Objective: Maximize profit **max  12.4 \* desk + 20.2 \* cell**

* Constraints:
    * The DeskProduction should be greater than or equal to 100.\
        **desk >= 100**       
    * The CellProduction should be greater than or equal to 100.\
        **Cell >= 100**
    * The assembly time for DeskProduction plus the assembly time for CellProduction should not exceed 400 hours.\
        **0.2 \* desk + 0.4 \* cell <= 401**
    * The painting time for DeskProduction plus the painting time for CellProduction should not exceed 490 hours.\
        **0.5 \* desk + 0.4 \* cell <= 492**    

In [ ]:
lm = Model(name='lp_telephone_production')
desk = lm.continuous_var(name='desk')
cell = lm.continuous_var(name='cell')
# write constraints
# constraint #1: desk production is greater than 100
lm.add_constraint(desk >= 100)

# constraint #2: cell production is greater than 100
lm.add_constraint(cell >= 100)

# constraint #3: assembly time limit
ct_assembly = lm.add_constraint( 0.2 * desk + 0.4 * cell <= 401)

# constraint #4: paiting time limit
ct_painting = lm.add_constraint( 0.5 * desk + 0.4 * cell <= 492)
lm.maximize(12.4 * desk + 20.2 * cell)

ls = lm.solve()
lm.print_solution()

objective: 20948.167
status: OPTIMAL_SOLUTION(2)
  desk=303.333
  cell=850.833


![img](https://camo.githubusercontent.com/b73348303cb92a1e7ee32a68d547205592af9e10/68747470733a2f2f69626d6465636973696f6e6f7074696d697a6174696f6e2e6769746875622e696f2f7475746f7269616c732f6a7570797465722f747261696e696e672f315f33392e706e673f7261773d74727565)

As we can see the optimal solution contains fractional values for number of telephones, which are not realistic. To ensure we get integer values in the solution, we can use integer decision variables.

Let's solve a new model, identical except that its two decision variables are declared as integer variables.

In [ ]:
im = Model(name='ip_telephone_production')
desk = im.integer_var(name='desk')
cell = im.integer_var(name='cell')
# write constraints
# constraint #1: desk production is greater than 100
im.add_constraint(desk >= 100)

# constraint #2: cell production is greater than 100
im.add_constraint(cell >= 100)

# constraint #3: assembly time limit
im.add_constraint( 0.2 * desk + 0.4 * cell <= 401)

# constraint #4: paiting time limit
im.add_constraint( 0.5 * desk + 0.4 * cell <= 492)
im.maximize(12.4 * desk + 20.2 * cell)

si = im.solve()
im.print_solution()

NameError: ignored

What if the question is a **binary** integer programming (0-1 type)? \
See: http://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_model.html

## Network modelling concepts

Any network structure can be described using two types of objects:

- Nodes (points, vertices): Defined points in the network, for example warehouses.
- Arcs (links, edges): An arc connects two nodes, for example a road connecting two warehouses. 

An arc can be _directed_, which means than an arc $a_{ij}$ from node $i$ to node $j$ is different from arc $a_{ji}$ that begins at node $j$ and ends at node $i$.

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/1_5.png?raw=true" >
</ul> 

 A sequence of arcs connecting two nodes is called a chain. Each arc in a chain shares exactly one node with the preceding arc.

 When all the arcs in a chain are directed such that it is possible to traverse the chain in the directions of the arcs from the start node to the end node, it is called a path.

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/1_6.png?raw=true" >
</ul> 


### The Transportation Problem

One of the most common real-world network problems is the transportation problem.  This type of problem involves a set of supply nodes and a set of demand nodes.  The objective is to minimize the transportation cost from the supply nodes to the demand nodes, so as to satisfy the demand, and without exceeding the suppliers’ capacities.  

Such a problem can be depicted in a graph, with supply nodes, demand nodes, and connecting arcs.  The supply capacity is indicated with the supply nodes, while the demand is indicated with the demand nodes, and the transportation costs are indicated on the arcs.  

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/1_8.png?raw=true" >
</ul> 

The LP formulation involves one type of variable, namely $x(i,j)$ representing the quantity transported from supply node $i$ to demand node $j$.  The objective is to minimize the total transportation cost across all arcs.  The constraints are flow conservation constraints.  The first two constraints state that the outflow from each supply node should be less than or equal to the supply capacity. The next three constraints state that the inflow into each demand node should equal the demand at that node. The domain for the shipments on the allowable arcs is set to be greater than or equal to zero, while the shipment quantities on the disallowed arcs are set to zero.  

Even though arcs $(1,4)$ and $(2,3)$ do not exist in the graph, the variables are included in the slide to show the special structure of the transportation problem.  If you were to formulate such a model in practice, you’d simply exclude these variables. 

#### Formulating a simple transportation problem with DOcplex

In the next section, we formulate the problem described above using DOcplex.

#### What data for the transpotation problem?

Input ndoes are integers ranging in {1, 2}; output nodes are integers ranging from 3 to 5.

The data consists in three Python dictionaries:

- one dictionary gives capacity values for all input nodes
- one dictionary contains demands for all target nodes
- one last dictionary holds cost values for some (source, target) pair of nodes.

In [ ]:
capacities = {1: 15, 2: 20}
demands = {3: 7, 4: 10, 5: 15}
costs = {(1,3): 2, (1,5):4, (2,4):5, (2,5):3}

# Python ranges will be used to iterate on source, target nodes.
source = range(1, 3) # {1,2}
target = range(3, 6) # {3,4,5}

#### Create a model instance

In [ ]:
from docplex.mp.model import Model

tm = Model(name='transportation')

#### Define the decision variables
- The continuous variable `x_i_j` represents the quantity transported from supply node `i` to demand node `j`.

In [ ]:
# create flow variables for each couple of nodes
# x(i,j) is the flow going out of node i to node j
x = {(i,j): tm.continuous_var(name='x_{0}_{1}'.format(i,j)) for i in source for j in target if (i,j) in costs}

tm.print_information()

Model: transportation
 - number of variables: 4
   - binary=0, integer=0, continuous=4
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP


#### Set up the constraints

- For each source node, the total outbound flow must be smaller than available quantity.
- For each target node, total inbound flow must be greater thand demand

In [ ]:
# for each node, total outgoing flow must be smaller than available quantity
for i in source:
    tm.add_constraint(tm.sum(x[i,j] for j in target if (i,j) in costs) <= capacities[i])
    
# for each target node, total ingoing flow must be greater thand demand
for j in target:
    tm.add_constraint(tm.sum(x[i,j] for i in source if (i,j) in costs) >= demands[j])

#### Express the business objective: minimize total flow cost

Each arc has a unit cost and we want to minimize the total cost. We only express cost when the arc exists.

In [ ]:
tm.minimize(tm.sum(x[i,j]*costs[i,j] for i in source for j in target if (i,j) in costs))

### Solve with the Decision Optimization solve service

If url and key are None, the Modeling layer will look for a local runtime, otherwise will use the credentials.

Look at the documentation for a good understanding of the various solving/generation modes.

If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage may fail and will need a paying subscription or product installation.

In any case, `Model.solve()` returns a solution object in Python, containing the optimal values of decision variables, if the solve succeeds, or else it returns `None`.

In [ ]:
tms = tm.solve()
tms.display()

solution for: transportation
objective: 114.000
status: OPTIMAL_SOLUTION(2)
x_1_3 = 7.000
x_1_5 = 5.000
x_2_4 = 10.000
x_2_5 = 10.000


## The Shortest path problem

The Shortest Path Problem is the problem of finding the shortest path through a network.  For example, to find the minimum travel time between two cities in a network of cities.  The shortest path problem is a special case of the transshipment problem, where there is exactly one supply node and one demand node, and the supply and demand are both equal to 1.  

<p>
<ul>
<img src = "https://ibmdecisionoptimization.github.io/tutorials/jupyter/training/1_12.png?raw=true" >
</ul> 

In this example, each node represents a city, and each arc represents the road connecting two cities.  The travel time is indicated on each arc.  The variable $x(i, j)$ takes a value of 1 if the arc between $i$ and $j$ is included in the shortest path, and zero otherwise.  The objective is to minimize the total travel time.  As with the other network problems, the constraints can be seen as flow conservation constraints.  A constraint exists for each node (or each city) and the constraints state that exactly one arc should be chosen into each city, and exactly one arc should be chosen out of each city.  

Again, even though the $x$ variables must take $0-1$ values, they can be declared as continuous due to the integrality property (that is, all the capacity and demand quantities are integer). 

### Model about the shortest path problem

Let us consider a directed graph $G = (V, A)$ with the set of nodes $V$ and the set of arcs $A$. Let $n$ and $m$ be the cardinality of $V$ and $A$, respectively. A path is a sequence of nodes $v_1, . . . , v_k$. We denote by $\delta^+(i)$ and $\delta^-(i)$ the set of outgoing and incoming arcs of node $i$. 

A standard integer programming formulation to determine a shortest path from node $s$ to node $t$ is the following:

<img src = "https://ftp.bmp.ovh/imgs/2020/12/49f35869d4f2b4f9.png" >

### Solve the problem with cplex

In [ ]:
# define the variables
cities = range(0,6) # {0, 1, 2, 3, 4, 5}
source = {0}
destination = {5}
distance = {(0,1): 2, (0,2): 4, (1,2): 1, (1,3): 2, (1,4): 4, (2,4): 2, (3,5): 3, (4,5): 1}


# create model
from docplex.mp.model import Model
im = Model(name='shortest_path_example')

# define decision variable x : binary for path included or not
x = {(i,j): im.binary_var(name='x_{0}_{1}'.format(i,j)) for i in cities for j in cities if (i,j) in distance}

In [ ]:
im.print_information()

Model: shortest_path_example
 - number of variables: 8
   - binary=8, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [ ]:
# constraint (2)
for i in source:
    im.add_constraint(im.sum(x[i,j] for j in cities if (i,j) in distance) - 
                      im.sum(x[j,i] for j in cities if (j,i) in distance ) == 1)

for i in destination:
    im.add_constraint(im.sum(x[i,j] for j in cities if (i,j) in distance) - 
                      im.sum(x[j,i] for j in cities if (j,i) in distance ) == -1)

for i in cities :
    if i not in source and i not in destination:
        im.add_constraint(im.sum(x[i,j] for j in cities if (i,j) in distance) - 
                          im.sum(x[j,i] for j in cities if (j,i) in distance ) == 0)

# constraint (3)
for i in cities :
    im.add_constraint(im.sum(x[i,j] for j in cities if (i,j) in distance) <= 1)

In [ ]:
# objective function (1)
im.minimize(im.sum(x[i,j]*distance[i,j] for i in cities for j in cities if (i,j) in distance))

In [ ]:
# get the solution
si = im.solve()
im.print_solution()

objective: 6
status: OPTIMAL_SOLUTION(2)
  x_0_1=1
  x_1_2=1
  x_2_4=1
  x_4_5=1


So the best path is : 
   > start --> city 1 --> city 2 --> city 4 --> end

##  Exercise 

### 1
A real estate development firm, Peterson and Johnson, is considering five possible development projects. The following table shows the estimated long-run profit (net present value) that each project would generate, as well as the amount of investment required to undertake the project, in units of millions of dollars.

| project number     | 1 | 2 | 3 | 4 | 5 |
|--------------------|---|---|---|---|---|
| Estimated profit   | 1 |1.8|1.6|0.8|1.4|
| Capital required   | 6 |12 |10 |4  | 8 | 

The owners of the firm, Dave Peterson and Ron Johnson, have raised \$20 million of investment capital for these projects. Dave and Ron now want to select the combination of projects that will maximize their total estimated long-run profit (net present value) without investing more that \\$ 20 million. \

Formulate and solve this problem (find best investment combination) with cplex.

In [ ]:
#define the variables
project = range(1, 6) # {1,2,3,4,5}
profit = {1: 1, 2: 1.8, 3:1.6, 4:0.8, 5:1.4}
capital = {1: 6, 2: 12, 3: 10, 4:4, 5:8}
investment = (20)


# create model
from docplex.mp.model import Model
im = Model(name='best-investment-combination')

# define decision variable x : binary for path included or not
x = {(i): im.binary_var(name='x_{0}'.format(i)) for i in project}

im.print_information()

Model: best-investment-combination
 - number of variables: 5
   - binary=5, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [ ]:
# constraint :  total capital required is equal or less than 20$
for i in project:
    im.add_constraint(im.sum(x[i]*capital[i] for i in project) <= investment)

# objective function (1)
im.maximize(im.sum(x[i]*profit[i] for i in project ))

In [ ]:
# get the solution
si = im.solve()
im.print_solution()

objective: 3.400
status: OPTIMAL_SOLUTION(2)
  x_1=1
  x_3=1
  x_4=1


### 2
Powerco has three electric power plants that supply the electric needs of four cities.

•The associated supply of each plant and demand of each city are
known.

•The cost of sending 1 million kwh of electricity from a plant to a
city depends on the distance the electricity must travel.

• Formulate an LP to minimize cost and solve this problem with cplex.

<p>
<ul>
<img src = "https://i.ibb.co/cCSMNhL/Screenshot-2021-12-16-at-23-20-17.png" >
</ul> 

In [ ]:
#define the variables
city = range(1, 5) # {1,2,3,4}
plant = range(1, 4) # {1,2,3}

cost = {(1,1): 8, (1,2): 6, (1,3):10, (1,4):9,(2,1): 9, (2,2): 12, (2,3):13, (2,4):7, (3,1): 14, (3,2): 9, (3,3):16, (3,4):5}
demand = {1: 45, 2: 20, 3:30, 4:30}
supply = {1:35, 2:50, 3:40}


# create model
from docplex.mp.model import Model
tm = Model(name='least-cost')

# define decision variable x : binary for path included or not
x = {(i,j): tm.continuous_var(name='x_{0}_{1}'.format(i,j)) for i in plant for j in city if (i,j) in cost}

tm.print_information()

Model: least-cost
 - number of variables: 12
   - binary=0, integer=0, continuous=12
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [ ]:
# constraint 1 : total outgoing supply must be equal or less than the available supply 
for i in plant:
    tm.add_constraint(tm.sum(x[i,j] for j in city if (i,j) in cost) <= supply[i])
# constraint 2 : each demand from a city must be equal or more than sum of supply
for j in city:
    tm.add_constraint(tm.sum(x[i,j] for i in plant if (i,j) in cost) >= demand[j])

# objective function (2)
tm.minimize(tm.sum(x[i,j]*cost[i,j] for i in plant for j in city if (i,j) in cost))



Model: least-cost
 - number of variables: 12
   - binary=0, integer=0, continuous=12
 - number of constraints: 7
   - linear=7
 - parameters: defaults
 - objective: minimize
 - problem type is: LP
solution for: least-cost
objective: 1020.000
status: OPTIMAL_SOLUTION(2)
x_1_2 = 10.000
x_1_3 = 25.000
x_2_1 = 45.000
x_2_3 = 5.000
x_3_2 = 10.000
x_3_4 = 30.000


In [ ]:
tm.print_information()
# get the solution
sol = tm.solve()
sol.display()

Model: least-cost
 - number of variables: 12
   - binary=0, integer=0, continuous=12
 - number of constraints: 7
   - linear=7
 - parameters: defaults
 - objective: minimize
 - problem type is: LP
solution for: least-cost
objective: 1020.000
status: OPTIMAL_SOLUTION(2)
x_1_2 = 10.000
x_1_3 = 25.000
x_2_1 = 45.000
x_2_3 = 5.000
x_3_2 = 10.000
x_3_4 = 30.000


### 3 The Shortest path problem
Solve the shortest path problem from $A$ to $F$ using cplex. ( be careful this is an undirected graph )


<p>
<ul>
<img src = "https://ftp.bmp.ovh/imgs/2020/12/bf64ba8bc7f61d16.png" >
</ul> 

In [ ]:
# define the variables
nodes = range(0,6) # {0, 1, 2, 3, 4, 5}
source = {0}
destination = {5}
distance = {(0,1): 8, (0,2): 10,
            (1,0): 8, (2,0): 10,
            (1,3):11, (1,4): 11, 
            (3,1):11, (4,1): 11, 
            (2,3): 7, (2,4): 9,
            (3,2): 7, (4,2): 9,  
            (3,5): 9, (4,5):8,
            (5,3): 9, (5,4):8}


# create model
from docplex.mp.model import Model
im = Model(name='shortest_undirected_path')

# define decision variable x : binary for path included or not
x = {(i,j): im.binary_var(name='x_{0}_{1}'.format(i,j)) for i in nodes for j in nodes if (i,j) in distance}

im.print_information()

Model: shortest_undirected_path
 - number of variables: 0
   - binary=0, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [ ]:
# constraint (2)
for i in source:
    im.add_constraint(im.sum(x[i,j] for j in nodes if (i,j) in distance) - 
                      im.sum(x[j,i] for j in nodes if (j,i) in distance ) == 1)

for i in destination:
    im.add_constraint(im.sum(x[i,j] for j in nodes if (i,j) in distance) - 
                      im.sum(x[j,i] for j in nodes if (j,i) in distance ) == -1)

for i in nodes :
    if i not in source and i not in destination:
        im.add_constraint(im.sum(x[i,j] for j in nodes if (i,j) in distance) - 
                          im.sum(x[j,i] for j in nodes if (j,i) in distance ) == 0)

# constraint (3)
for i in nodes :
    im.add_constraint(im.sum(x[i,j] for j in nodes if (i,j) in distance) <= 1)

In [ ]:
# objective function (3)
im.minimize(im.sum(x[i,j]*distance[i,j] for i in nodes for j in nodes if (i,j) in distance))

sol = im.solve()
sol.display()

solution for: shortest_undirected_path
objective: 26
status: OPTIMAL_SOLUTION(2)
x_0_2 = 1
x_2_3 = 1
x_3_5 = 1


### 4 Vertex Coloring 

Vertex coloring is the most common graph coloring problem. The problem is, given $m$ colors, find a way of coloring the vertices of a graph such that **no two adjacent vertices are colored using same color**. The most common type of vertex coloring seeks to **minimize the number of colors for a given graph**. Such a coloring is known as a minimum vertex coloring, and the minimum number of colors which with the vertices of a graph G may be colored is called the chromatic number.

Calculate the chromatic number of the following graph using cplex.

<p>
<ul>
<img src = "https://ftp.bmp.ovh/imgs/2020/12/5e4cf3c25b43b0b6.png" >
</ul> 



In [ ]:
# define the variables
nodes = range(0,6) # {0, 1, 2, 3, 4, 5}
arcs = {(0,4), (0,2), (0,5),
        (1,4),(1,5),
        (2,0),(2,3),(2,4),
        (3,2),
        (4,0),(4,1),(4,5),(4,2),
        (5,1), (5,0), (5,4)}
colors = range(0,6) # {0, 1, 2, 3, 4, 5}

# create model
from docplex.mp.model import Model
vertex = Model(name='vertex-coloring')

# variable m : i and j adjacent
m = {(i,j): vertex.binary_var(name='m_{0}_{1}'.format(i,j)) for i in nodes for j in nodes if (i,j) in arcs}

# variable x : i actif color
x = {(i): vertex.binary_var(name='x_{0}'.format(i)) for i in colors} 

# variable y : if the color i is affected to a node j

y = {(i,j): vertex.binary_var(name='y_{0}_{1}'.format(i,j)) for i in colors for j in nodes}

vertex.print_information()

Model: vertex-coloring
 - number of variables: 58
   - binary=58, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [ ]:
# constraint 1 : unique color for each node
for j in nodes:
  vertex.add_constraint(vertex.sum(y[i,j] for i in colors ) == 1)

# constraint 2 : actif color must be affected to a node, et vice versa
for i in colors : 
  for j in nodes :
    vertex.add_constraint((y[i,j] - x[i] ) <= 0)

# constraint 3 : adjacents nodes must have different colors

for j in colors :
  for a in nodes : 
    for b in nodes :
       if (a,b) in arcs : 
        vertex.add_constraint(((y[j,a]+y[j,b])) <= 1)


In [ ]:
# objective function (4)
vertex.minimize(vertex.sum(x[i] for i in colors))

sol = vertex.solve()
sol.display()

solution for: vertex-coloring
objective: 3
status: OPTIMAL_SOLUTION(2)
x_0 = 1
x_2 = 1
x_5 = 1
y_0_4 = 1
y_2_2 = 1
y_2_5 = 1
y_5_0 = 1
y_5_1 = 1
y_5_3 = 1


> Email me your solutions before 23:59 on 23 December) with mail title **\<M1-MIAGE\> RO TP2 prenom.nom** to **shangyuan.zhang@universite-paris-saclay.fr**

>The solution should be a **.pdf** or **.html** file with title **prenom_nom_TP2.pdf (.html)** generated by this notebook. (see how to generate .pdf or .html from the image below)


>For any problem, see : https://stackoverflow.com/questions/15998491/how-to-convert-ipython-notebooks-to-pdf-and-html/25942111

<div>
<img src="https://i.stack.imgur.com/vAjri.png" width="600"/>
</div>

In [ ]:
''' A title/file name generator''' 

prenom = 'dounia'  # type your first name here
nom = 'bouloudene'    # type your last name here
file_type = '.pdf'
# file_type = '.html'

print("your email title : \n" " <M1-MIAGE> RO TP2 "+ prenom + "." + nom)
print("\nyour file name : \n" + prenom + "_" + nom + "_TP2" + file_type )

your email title : 
 <M1-MIAGE> RO TP2 dounia.bouloudene

your file name : 
dounia_bouloudene_TP2.pdf
